# Acquisition

In [1]:
PARAMS = {
    'session_name':'test2',
    'working_directory':'synctest',
    'recording_length':1200,
    'frame_width':640,
    'frame_height':480,
    'fps': 30,
}

In [2]:
import shutil, json, os, sys, time, pickle, h5py, cv2, matplotlib.pyplot as plt, numpy as np
sys.path.append('/usr/local/lib/')
import pyrealsense2 as rs
from MoseqMulti_acquire import *

### Create data directory

In [3]:
if not os.path.exists(PARAMS['working_directory']): os.makedirs(PARAMS['working_directory'])
if not os.path.exists(PARAMS['working_directory']+'/data/'): os.makedirs(PARAMS['working_directory']+'/data/')

### List currently plugged in devices

In [5]:
serial_numbers = get_connected_devices()

Found device  <pyrealsense2.device: Intel RealSense D415 (S/N: 816312062021)>
Found device  <pyrealsense2.device: Intel RealSense D415 (S/N: 816312062005)>
Found device  <pyrealsense2.device: Intel RealSense D415 (S/N: 739112061202)>
Found device  <pyrealsense2.device: Intel RealSense D415 (S/N: 739112060301)>
Found device  <pyrealsense2.device: Intel RealSense D415 (S/N: 816312061973)>


### Create a pipeline for each plugged in camera

In [6]:
pipelines, configs = get_pipelines(serial_numbers, PARAMS)

### Actually do the recording (saved to bag files)

In [7]:
profiles = start_pipes(pipelines,configs,serial_numbers) 
intrinsics = get_intrinsics(pipelines)

# actually do the recording
if len(serial_numbers)==0:
    print('No cameras connected')
else:
    try:        
        print('Started recording')
        total_frames = 0
        timestamps = {n:[] for n in serial_numbers}
        start_time = time.time()
        while time.time()-start_time < PARAMS['recording_length']:
            if ((total_frames) % int(1000/30) == 0): 
                print('\rRecorded',int(time.time()-start_time+1),'/', PARAMS['recording_length'], 'seconds', end='')
            for n in pipelines: 
                frames = pipelines[n].wait_for_frames()
                if total_frames < 100: timestamps[n].append(rs.frame.get_timestamp(frames))
            total_frames += 1

    # stop recording and save metadata
    finally: 
        stop_time = time.time()
        stop_pipes(pipelines)
        metadata = get_metadata(serial_numbers,start_time,stop_time,total_frames, intrinsics, timestamps, PARAMS)
        json.dump(metadata, open(PARAMS['working_directory']+'/data/'+PARAMS['session_name']+'_metadata.json','w'))
        print('\nFinished recording '+repr(total_frames)+' frames in '+repr(stop_time-start_time)+' seconds')

Started recording
Recorded 1200 / 1200 seconds
Finished recording 35889 frames in 1200.0096361637115 seconds


In [8]:
# need to delete these or else the .bag file won't properly close
del pipelines
del configs
del profiles